In [1]:
import numpy as np
import pandas as pd
import random as rd
import datetime
import os
import itertools

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [2]:
# Define some constants that we will use
PROJECT_PATH = '/Users/chris/programming/kaggle-predict-future-sales'

TRAIN = 'train'
VALID = 'valid'
TEST = 'test'

In [3]:
# Read in raw CSV files

csv_data = dict()

# Import all csv file data
csv_data[ 'sales_daily' ] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/sales_train.csv') )
csv_data['item_cat'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/item_categories.csv') )
csv_data['item'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/items.csv') )
csv_data['sub'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/sample_submission.csv') )
csv_data['shops'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/shops.csv') )
csv_data['test'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/test.csv') )

In [4]:
# Add some new columns and aggregate from daily to monthly
############################################################

# Add the daily revenue
csv_data['sales_daily'][ "revenue" ] = csv_data['sales_daily'].item_price * csv_data['sales_daily'].item_cnt_day

# Aggregate the monthly data
agg_rules = {'item_price' : "mean", "revenue" : "sum", "item_cnt_day" : "sum" }
groupby_cols = [ 'date_block_num', "item_id", "shop_id" ]

# Add the effective item price
csv_data['sales_monthly'] = csv_data['sales_daily'].groupby( groupby_cols ).agg( agg_rules ).reset_index()

# Rename the column to reflect monthly data
csv_data['sales_monthly'].rename( columns={ 'item_cnt_day' : 'item_cnt_month'}, inplace=True )

# Add unit price as a column
csv_data['sales_monthly']['item_price_unit'] = np.round( csv_data['sales_monthly']['revenue'] / \
                              ( 1e-6 + csv_data['sales_monthly']['item_cnt_month'] ) )


In [43]:
# Get all combinations of shops/items for each date
############################################################

df_monthly = csv_data['sales_monthly']
dates = df_monthly['date_block_num'].unique()

df_id = pd.DataFrame( [], columns=['date_block_num', 'shop_id', 'item_id'])
for dt in dates:
        
    df_t = df_monthly[ df_monthly['date_block_num'] == dt ]
    uniq_shops = df_t['shop_id'].unique()    
    uniq_items = df_t['item_id'].unique()
        
    new_rows = pd.DataFrame( itertools.product( [dt], uniq_shops, uniq_items ), columns=df_id.columns )
    df_id = pd.concat( [ df_id, new_rows ], sort=False, axis=0 )

# Join the test IDs to the data frame
df_t = csv_data['test'].copy()
df_t['date_block_num'] = 1 + df_id['date_block_num'].max()
df_id = pd.concat( [ df_id, df_t ], sort=False, axis=0 ).drop(['ID'], axis=1)

# Create a data frame using all the shop/item pairs from each month
df_sales = df_id.merge( csv_data['sales_monthly'], on=['date_block_num', 'shop_id', 'item_id' ], how='left' )

# Set missing values for revenue and item count to 0
df_sales.loc[ np.isnan(df_sales['revenue']), ['revenue'] ] = 0
df_sales.loc[ np.isnan(df_sales['item_cnt_month']), ['item_cnt_month'] ] = 0

In [44]:
# Add new columns to the data frame
############################################################

# Add the category id
df_sales = df_sales.merge( csv_data['item'], on='item_id' )
df_sales = df_sales.drop('item_name', axis=1 )

# Add a column combining shop and item, which together with date_block_num is a unique id
df_sales['shop_item_id'] = df_sales['shop_id'] + 100 * df_sales['item_id']

# Add the month and year
df_sales['month'] = 1 + (df_sales['date_block_num'] % 12)
df_sales['year'] = 2013 + (df_sales['date_block_num'] // 12)

# Add the dates
dates = pd.Series( [ datetime.date( y, m, 15 ) for y in range(2013,2016) for m in range(1, 13) ], \
                          index=range(0,36) )
df_sales['date'] = df_sales['date_block_num'].map(dates)

# Rename the target column and make it the first column
target = df_sales.loc[:,['item_cnt_month']]
target.columns = [ 'TARGET']
df_sales = pd.concat( [ target, df_sales.drop('item_cnt_month', axis=1) ], axis=1 ) 

In [84]:
# Define functions to help creating pivot tables

def create_pivot_ts( input_table, pivot_column, val_column, agg_rule, fill_method, min_date_block_num=0 ):
    
    df = input_table[ input_table['date_block_num'] >= min_date_block_num ]
    
    if isinstance( pivot_column, str ):
        pivot_columns = [ pivot_column ]
    else:
        pivot_columns = pivot_column
    
    # Make time series out of the monthly  sales
    tmp_table = df.groupby( [ 'date_block_num' ] + pivot_columns ).agg( { val_column : agg_rule } ).reset_index()
    ts = tmp_table.pivot_table( val_column, index="date_block_num", columns=pivot_columns )

    # Fill missing values with 0
    if fill_method == "zero":
        ts = ts.fillna(0)
    elif fill_method == 'ffill':
        ts = ts.fillna(method=missing_method)
    elif fill_method != 'none':
        ValueError( 'Unsupported value: .' + missing_method )

    # Set negative values to 0
    ts[ ts < 0 ] = 0

    # Make sure the dates are sorted
    ts.sort_index(axis=0, ascending=True, inplace=True )
    
    return(ts)

In [ ]:
# Create a pivot table with shop/item combinations and monthly sales
ts_sales_raw = create_pivot_ts( df_sales, pivot_column=[ 'shop_item_id' ], \
    val_column='TARGET', agg_rule='sum', fill_method='none', min_date_block_num=0 )
ts_sales = ts_sales_raw.fillna(0)

# The last date is just test data, so we don't know the feature values
ts_sales_raw.iloc[-1,:] = np.nan
ts_sales.iloc[-1,:] = np.nan

In [17]:
# Create a pivot table with shop/item combinations and monthly sales
ts_sales_raw = create_pivot_ts( df_sales, pivot_column=[ 'shop_id', 'item_id', 'item_category_id' ], \
    val_column='item_cnt_month', agg_rule='sum', fill_method='none', min_date_block_num=0 )
ts_sales = ts_sales_raw.fillna(0)

# The last date is just test data, so we don't know the feature values
ts_sales_raw.iloc[-1,:] = np.nan
ts_sales.iloc[-1,:] = np.nan

In [18]:
# Create a pivot table with shop/item combinations and monthly revenues
ts_revenue_raw = create_pivot_ts( df_sales, pivot_column=[ 'shop_id', 'item_category_id', 'item_id' ], \
    val_column='revenue', agg_rule='sum', fill_method='none', min_date_block_num=0 )
ts_revenue = ts_revenue_raw.fillna(0)

# The last date is just test data, so we don't know the feature values
ts_revenue_raw.iloc[-1,:] = np.nan
ts_revenue.iloc[-1,:] = np.nan

In [106]:
# Initialize a list to store features. We will concatenate these afterwards with hstack
feature_vals = []
feature_names = []
cat_features = []

# Just keep the last M months of observations, and convert to a numpy column vector
months_to_keep = 10
process_features = lambda x : x.iloc[-months_to_keep:,:].to_numpy().ravel()[:,np.newaxis]

# Get the target - monthly sales, the next month into the future`
ts = ts_sales.shift(periods=-1)
feature_vals.append( process_features(ts) )
feature_names.append( 'TARGET' )

# Get lagged monthly sales
print( 'Calculating lagged sales...')
lags = [1, 2, 3, 4, 5, 6, 9, 12]
for L in lags:
    ts = ts_sales.shift(periods=L-1)
    feature_vals.append( process_features(ts) )
    feature_names.append( 'sales_lag_{:02}'.format(L) )

# Get lagged monthly mean sales
print( 'Calculating rolling mean sales...')
means = [2, 3, 4, 5, 6, 9, 12]
for M in means:
    ts = ts_sales.rolling(window=M).mean()
    feature_vals.append( process_features(ts) )    
    feature_names.append( 'sales_mean_{:02}'.format(M) )

# Get 12-month standard deviation
print( 'Calculating std. dev. of sales...')
ts = ts_sales.rolling(window=12).std()
feature_vals.append( process_features(ts) )
feature_names.append( 'sales_std_12' )

# Get the 12-month quartiles
print( 'Calculating quantiles of sales...')
for p in [ 0.25, 0.50, 0.75]:
    ts = ts_sales.rolling(window=12).quantile(p)
    feature_vals.append( process_features(ts) )    
    feature_names.append( 'sales_percentile_{:02}'.format(int(p * 100) ) )

# Add some categorical features
print( 'Calculating categorical features...')
for j, col in enumerate(ts_sales.columns.names):
    ts = np.array([ x[j] for x in ts_sales.columns ] * ts_sales.shape[0]).reshape( ts_sales.shape[0], ts_sales.shape[1])
    feature_vals.append( process_features(pd.DataFrame(ts) ) )  
    feature_names.append( col )
    cat_features.append(col)

# Add some more categorical features for the date block, year and month
ts_dates = np.tile( np.array(ts_sales.index)[:,np.newaxis], ts_sales.shape[1] )
feature_vals.append( process_features(pd.DataFrame(ts_dates) ) )
feature_vals.append( process_features(pd.DataFrame( 1 + ( ts_dates % 12 ) ) ) )
feature_vals.append( process_features(pd.DataFrame( 2013 + ( ts_dates // 12 ) ) ) )
feature_names = feature_names + [ 'date_block_num', 'month', 'year' ]
cat_features = cat_features + [ 'date_block_num', 'month', 'year' ]    

Calculating lagged sales...
Calculating rolling mean sales...
Calculating std. dev. of sales...
Calculating quantiles of sales...
Calculating categorical features...


In [107]:
def get_months_since_first_and_last_observation( input_ts ):
    """Gets the number of months since the first and last observation in each columns, 
    at each point in time. This function has no 'look-forward' bias."""
    
    # Create an array with the index (0 to T) of any non-zeros entries
    month_of_obs = input_ts.to_numpy() * np.arange(1,input_ts.shape[0]+1)[:,np.newaxis]
    month_of_obs = month_of_obs.astype('float32')
    month_of_obs[ month_of_obs < 0.01 ] = np.nan

    # Make a data frame
    df_month_of_obs = pd.DataFrame( month_of_obs, columns=input_ts.columns)

    # Loop through the time steps and find the months since first/last action at each t
    months_since_first_obs = np.nan * np.ones_like(month_of_obs)
    months_since_last_obs = np.nan * np.ones_like(month_of_obs)
    for t in range( month_of_obs.shape[0] ):
        if t % 10 == 0:
            print(t)
        months_since_first_obs[t,:] = t - df_month_of_obs.iloc[:(t+1),:].idxmin(axis=0)
        months_since_last_obs[t,:] = t - df_month_of_obs.iloc[:(t+1),:].idxmax(axis=0)    

    # Fill NaN's with 999 for shop/items that have never seen a sale
    months_since_first_obs[ np.isnan( months_since_first_obs ) ] = 999
    months_since_last_obs[ np.isnan( months_since_last_obs ) ] = 999

    # Convert back into pandas Dataframe
    months_since_first_obs = pd.DataFrame( months_since_first_obs, columns=ts_sales.columns )
    months_since_last_obs = pd.DataFrame( months_since_last_obs, columns=ts_sales.columns )
    
    return months_since_first_obs, months_since_last_obs

In [108]:
# Get the number of months since the first and last sale, at each point in time
months_since_first_sale, months_since_last_sale = \
        get_months_since_first_and_last_observation( ts_sales )

/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  


0
10
20
30


In [109]:
# Add to features
feature_vals.append( process_features( months_since_first_sale ) )
feature_names.append( 'months_since_first_sale' )
feature_vals.append( process_features( months_since_last_sale ) )
feature_names.append( 'months_since_last_sale' )

In [110]:
sales_total_by_shop = ts_sales_raw.groupby( level='shop_id', axis=1).transform('sum')
sales_total_by_item = ts_sales_raw.groupby( level='item_id', axis=1).transform('sum')
sales_total_by_cat = ts_sales_raw.groupby( level='item_category_id', axis=1).transform('sum')

In [111]:
sales_count_by_shop = ts_sales_raw.groupby( level='shop_id', axis=1).transform('count')
sales_count_by_item = ts_sales_raw.groupby( level='item_id', axis=1).transform('count')
sales_count_by_cat = ts_sales_raw.groupby( level='item_category_id', axis=1).transform('count')

In [119]:
df_sales.groupby( [ 'date_block_num', 'shop_id' ] )['item_id'].transform('sum')

(1823324,)

214200

In [128]:
# Join the features into a pandas data frame
features_list = [ x for x in feature_vals ]
df_full = pd.DataFrame( np.hstack(features_list), columns=feature_names)

# Remove all rows with NA's (except when they occur in the TARGET column)
na_locs = df_full.isna()
df_full = df_full[~np.any(na_locs.iloc[:,1:], axis=1 ) ]

In [129]:
# Downcast the categorical features to int32, and other columns to float32

for col in list(df_full.columns):
    if col in cat_features:
        df_full[col] =  df_full[col].astype('int32')
    else:
        df_full[col] =  df_full[col].astype('float32')

In [130]:
# Save the raw features to a HDF5 file
file_name = os.path.join( PROJECT_PATH, 'preprocessed_data/raw_features.h5')
df_full.to_hdf(file_name, key='raw_features')     

In [ ]:
# Read in the raw features from a HDF5 file
#cat_features = [ 'shop_id', 'item_id', 'date_block_num', 'month', 'year']
# file_name = os.path.join( PROJECT_PATH, 'preprocessed_data/raw_features.h5')
# df_full = pd.read_hdf(file_name, key='raw_features')

In [246]:
# In the test set, just keep shop/item combinations required in the output
df_test = df_full.merge( csv_data['test'], on=['shop_id', 'item_id'], how='left' )
df_test = df_test[ ~np.isnan(df_test['ID'] ) ]
df_test.sort_values( [ 'date_block_num', 'ID' ], inplace=True )

In [258]:
dt = data_sets[-1][-2]
fs = np.count_nonzero( dt['months_since_first_sale'] == 999 ) / len(dt)
print( 'Months since first sale: {}'.format(fs) )
tmp = dt['months_since_first_sale'][ dt['months_since_first_sale'] != 999 ].mean()
print( 'Avg. months since first sale: {}'.format(tmp) )
tmp = dt['months_since_first_sale'][ dt['months_since_first_sale'] != 999 ].median()
print( 'Median months since first sale: {}'.format(tmp) )

fs = np.count_nonzero( dt['months_since_last_sale'] == 999 ) / len(dt)
print( 'Months since last sale: {}'.format(fs) )
tmp = dt['months_since_last_sale'][ dt['months_since_last_sale'] != 999 ].mean()
print( 'Avg. months since last sale: {}'.format(tmp) )
tmp = dt['months_since_last_sale'][ dt['months_since_last_sale'] != 999 ].median()
print( 'Median months since last sale: {}'.format(tmp) )

Months since first sale: 0.479906629318394
Avg. months since first sale: 15.133522987365723
Median months since first sale: 13.0
Months since last sale: 0.479906629318394
Avg. months since last sale: 5.357895374298096
Median months since last sale: 2.0


In [283]:
np.isnan( ts_sales_raw ).sum(axis=0)
#df_agg = csv_data['sales_monthly'].groupby( ['shop_id', 'item_id' ]).sum()
len(df_agg) / ( len(csv_data['sales_monthly']['item_id'].unique()) * len(csv_data['sales_monthly']['shop_id'].unique()) )



0.32414973785176016

In [288]:
# Get different datasets, in chronological order
# The test set is always the next period after the end of the train set.
##########################################################################

def get_validation_set( df, idx, min_train_size=4 ):

    date_blocks = df['date_block_num']
    uniq_date_blocks = np.sort( date_blocks.unique() )

    n_datasets = len(uniq_date_blocks) - min_train_size - 1
    if idx >= 0:
        split_date = uniq_date_blocks[min_train_size + idx]
    else:
        split_date = uniq_date_blocks[idx]    

    xtrain = df[ date_blocks < split_date].drop('TARGET', axis=1 )
    ytrain = df[ date_blocks < split_date ]['TARGET'][:,np.newaxis]
    xtest = df[ date_blocks == split_date ].drop('TARGET', axis=1 )
    ytest = df[ date_blocks == split_date ]['TARGET'][:,np.newaxis]

    return xtrain, ytrain, xtest, ytest


##########################################################################
# Construct some train/validation/test data sets
data_sets = []
for j in range(100):
    try:
        xtrain, ytrain, xtest, ytest = get_validation_set( df_full, j, min_train_size=4 )
        data_sets.append( ( xtrain, ytrain, xtest, ytest ) )
    except IndexError:
        break

# Replace the last validation set with the test set
_, _, xtest, ytest = get_validation_set( df_test.drop('ID', axis=1), j-1, min_train_size=4 )
data_sets[-1] = ( data_sets[-1][0], data_sets[-1][1], xtest, ytest )

In [133]:
def write_forecast_to_file( yhat, X_test, output_file ):
    
    if isinstance( yhat, pd.Series ):
        yhat = yhat.to_numpy()
        
    # Add the shop and item ids to the forecast so we can merge it with the test csv
    yhat_mtx = np.hstack( [ yhat.reshape(len(yhat),1), X_test[['shop_id', 'item_id']].to_numpy() ] )
    df_fcst = pd.DataFrame( yhat_mtx, columns=['item_cnt_month', 'shop_id', 'item_id'] )

    # Merge the forecast with the test csv
    df_out = csv_data['test'].merge( df_fcst, on=['shop_id', 'item_id'], suffixes=('_test', '_forecast'), how='left' )
    df_out = df_out[['ID', 'item_cnt_month']]

    # Write the combined csv to file
    df_out.to_csv( output_file, index=False )
    return df_out


def get_results_file_name(model_name, idx, data_sets):
    
    if len(data_sets) == 1 + idx:
        output_folder = 'test'
    else:
        output_folder = 'validation_{}'.format(idx)
        
    output_file = os.path.join( PROJECT_PATH, 'results', output_folder, model_name + '.csv')    
    
    return(output_file)

In [134]:
from sklearn.linear_model import LinearRegression, Ridge

def fit_model( base_model_constructor, model_name, data_sets, \
              feature_names=[], forecast_range=(-np.inf, +np.inf) ):

    fitted_models = []
    for idx in range(len(data_sets)):

        model = base_model_constructor()
        X_train_full, y_train, X_test_full, y_test = data_sets[idx]

        if len(feature_names) == 0:
            X_train = X_train_full
            X_test = X_test_full
        else:
            X_train = X_train_full[feature_names]
            X_test = X_test_full[feature_names]
        
        # Fit the model on the train data and make a prediction on the test set
        model.fit( X_train, y_train )
        y_hat = model.predict(X_test)
        
        # Clip forecasts to be in the desired range
        y_hat = np.maximum( forecast_range[0], np.minimum( forecast_range[1], y_hat ) )

        # Calculate the RMSE
        if idx == len(data_sets) - 1:
            rmse = np.nan
        else:
            rmse = np.sqrt( mean_squared_error( y_hat, y_test ) )
            
        print( 'Writing results for data set {}. RMSE is {}'.format(idx, rmse ) )

        # Write the output to file
        output_file = get_results_file_name(model_name, idx, data_sets)        
        write_forecast_to_file( y_hat, X_test_full, output_file )
        
        fitted_models.append(model)
        
    return fitted_models

In [135]:
model_name = 'linreg_01'
base_model = lambda : LinearRegression()
fitted_models = fit_model( base_model, model_name, data_sets )

Writing results for data set 0. RMSE is 0.7887855760830847
Writing results for data set 1. RMSE is 0.8457239282536244
Writing results for data set 2. RMSE is 4.165847882776679
Writing results for data set 3. RMSE is 3.363813572953488
Writing results for data set 4. RMSE is nan


In [136]:
model_name = 'linreg_02'
base_model = lambda : LinearRegression()
fitted_models = fit_model( base_model, model_name, data_sets, forecast_range=(0,500) )

Writing results for data set 0. RMSE is 0.7615322699064294
Writing results for data set 1. RMSE is 0.8412637628452622
Writing results for data set 2. RMSE is 4.163945612603016
Writing results for data set 3. RMSE is 3.3460046410851687
Writing results for data set 4. RMSE is nan


In [301]:
class Benchmark():
    
    def __init__(self, feature_name ):
        self.feature_name = feature_name
        
    def fit(self, X, y ):
        pass
    
    def predict(self, X):
        yhat = np.maximum( 0, np.minimum( 20, X[self.feature_name] ) )
        return( yhat )
    
class LinReg():
        
    def fit(self, X, y ):
        self.model = LinearRegression()
        #model = Ridge(alpha=0.2, normalize=True)
        self.model.fit( X, y )
    
    def predict(self, X):
        # Run the model on the validation set and see the score
        yhat = self.model.predict( X )
        yhat = np.minimum( 20, np.maximum(0, yhat) )       
        
        # Set the expected sales equal to 0 for shop/items that have not been seen before
        idx = X['months_since_first_sale'] == 999
        yhat[idx] = 0.3
        return yhat

In [298]:
model_name = 'benchmark'
base_model = lambda : Benchmark( 'sales_lag_01')
fitted_models = fit_model( base_model, model_name, data_sets, forecast_range=(0,20) )

Writing results for data set 0. RMSE is 1.2838308811187744
Writing results for data set 1. RMSE is 1.2979270219802856
Writing results for data set 2. RMSE is 4.297789573669434
Writing results for data set 3. RMSE is 3.5371546745300293
Writing results for data set 4. RMSE is nan


In [296]:
model_name = 'linreg_03'
base_model = lambda : LinReg()
fitted_models = fit_model( base_model, model_name, data_sets, forecast_range=(0,20) )

Writing results for data set 0. RMSE is 1.2713474990173441
Writing results for data set 1. RMSE is 1.2771579664777808
Writing results for data set 2. RMSE is 4.3283066851969325
Writing results for data set 3. RMSE is 3.5152167824381078
Writing results for data set 4. RMSE is nan


In [299]:
np.mean( fitted_models[-1].predict( data_sets[-1][-2]) )

0.25564894

In [146]:
from sklearn.decomposition import NMF

X_train, y_train, X_test, y_test = data_sets[0]
target_features = X_train.columns[:-5]

nmf = NMF(n_components=3)
nmf.fit(X_train[target_features])

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=3, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [179]:
from sklearn.neighbors import NearestNeighbors
X_train, y_train, X_test, y_test = data_sets[0]

model = NearestNeighbors(n_neighbors=3, n_jobs=8)
N = 500000

x_nmf_train = nmf.transform(X_train[target_features])[:N,:]
x_nmf_test = nmf.transform(X_test[target_features])[:N,:]

model.fit(x_nmf_train, y_train)
#y_hat_0 = model.

#rmse = np.sqrt( mean_squared_error( y_hat_0, y_test ))
#print( 'RMSE: {}'.format(rmse) )

CPU times: user 2min 47s, sys: 2.42 s, total: 2min 49s
Wall time: 2min 48s


30.0

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_name = 'knn_01'
feature_names = X_train.columns[1:-6]

base_model = lambda : NearestNeighbors(n_neighbors=5)
fitted_models = fit_model( base_model, model_name, data_sets, feature_names=feature_names )

In [145]:
bmk_1 = pd.read_csv( os.path.join( PROJECT_PATH, 'results/test/benchmark.csv' ) )
bmk_0 = pd.read_csv( os.path.join( PROJECT_PATH, 'forecasts/test/benchmark.csv' ) )
bmk_0.sort_values('ID', inplace=True)

out = bmk_0.merge(bmk_1, on='ID', suffixes=('_0', '_1'))
out.columns = [ 'ID', 'old', 'new']

np.corrcoef( out['old'], out['new'])

array([[1., 1.],
       [1., 1.]])

ID                107099.50000
item_cnt_month         0.18409
dtype: float64

In [ ]:
np.sum( ma_ts.to_numpy() > 0, axis=1 ) / np.sum( ts_sales.to_numpy(), axis=1 )[:-1]

In [ ]:
import collections
ctr = collections.Counter( bmk_0['item_cnt_month'] )
ctr 

In [ ]:

sales_cols = ['sales_lag_01', 'sales_lag_02', 'sales_lag_03', 'sales_lag_04', 'sales_lag_05', 'sales_lag_06', 'sales_lag_09', 'sales_lag_12' ]
for L in range(len(data_sets)):
    X_train, _, _, _ = data_sets[L]
    for col in sales_cols:
        ctr = collections.Counter(X_train[col])
        N = X_train.shape[0]
        cnt_vals = np.array([ np.log(ctr[x]/N) for x in range(19) ])
        x_vals = np.arange(0,19)
        plt.plot( x_vals, cnt_vals )

              

In [ ]:
cnt_vals = np.array([ np.log( np.log(ctr[x]) ) for x in range(19) ])
x_vals = np.arange(0,19)
plt.plot( x_vals, cnt_vals )

model = LinearRegression()
model.fit( x_vals[:,np.newaxis], cnt_vals[:,np.newaxis] )
alpha = model.intercept_[0]
beta = model.coef_[0][0]
y_hat = alpha + beta * x_vals
plt.plot( x_vals, y_hat )


In [ ]:

def get_rmse( model_name, data_sets )

    rmse = []
    for idx in range(len(data_sets) - 1):

        output_file = get_results_file_name(model_name, idx, data_sets)
        y_hat = pd.read_csv(output_file)
        _, _, X_test, y_test = data_sets[idx]
        test = pd.concat( [ X_test.reset_index(drop=True), pd.DataFrame( y_test, columns=['TARGET']) ], axis=1 )

        joined = y_hat.merge( test, on='ID' )

        rmse.append( np.sqrt( mean_squared_error( joined['TARGET'], joined['item_cnt_month'] ) ) )
        
        return(rmse)

In [ ]:
rmse = []
for j in range(len(data_sets)-1):
    X_train, y_train, X_test, y_test = data_sets[j]
    rmse.append( np.sqrt(mean_squared_error( np.minimum( 20, X_train['sales_lag_01'] ), y_train ) ) )

print(rmse)
print(np.mean(rmse))    

In [ ]:
# Generate the benchmark forecast
###################################

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-2]

# Use the previous month's sales as the benchmark forecast
yhat = X_test['sales_lag_01']

# Cap the forecast values at 20, and floor it at 0
yhat = np.maximum( np.minimum( yhat, 20 ), 0 )

rmse = np.sqrt( mean_squared_error( yhat, y_test ) )
print( 'RMSE on the validation set: {}'.format(rmse))

###################################################
# Now find the forecast for the test set
###################################################

# Get the last set of features before the test period
_, _, X_test, _ = data_sets[-1]

# Use the previous month's sales as the benchmark forecast
yhat = X_test['sales_lag_01']

# Cap the forecast values at 20, and floor it at 0
yhat = np.maximum( np.minimum( yhat, 20 ), 0 )

# Write the data to file
#output_file = '../forecasts/test/benchmark.csv'
#df_out = write_forecast_to_file( yhat, X_test, output_file )

In [ ]:
# Try a forecast with linear regression
###################################################

from sklearn.linear_model import LinearRegression, Ridge

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-2]

# Fit the model using th
model = LinearRegression()
#model = Ridge(alpha=0.2, normalize=True)
model.fit( X_train, y_train )

# Run the model on the validation set and see the score
yhat = model.predict( X_test )
yhat = np.minimum( 20, np.maximum(0, yhat) )

rmse = np.sqrt( mean_squared_error( yhat, y_test ) )
print( 'RMSE on the validation set: {}'.format(rmse))


In [ ]:
# Try a forecast with Ridge regression
###################################################

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-2]

# Fit the model using th
model = Ridge(alpha=2, normalize=True)
model.fit( X_train, y_train )

# Run the model on the validation set and see the score
yhat = model.predict( X_test )
yhat = np.minimum( 20, np.maximum(0, yhat) )

rmse = np.sqrt( mean_squared_error( yhat, y_test ) )
print( 'RMSE on the validation set: {}'.format(rmse))

In [208]:
# Try a forecast with LightGBM
###################################################

import lightgbm as lgb

class LGBM():
    
    def __init__(self, params, categorical_feature ):
        self.params = params
        self.categorical_feature = categorical_feature
        self.model = []
        
    def fit(self, X_train, y_train, X_test=None, y_test=None ):
        lgb_train = lgb.Dataset(X_train, label=y_train.ravel(), \
                            categorical_feature=self.categorical_feature )
        
        if y_test is None or np.all( np.isnan(y_test) ):
            # If there is no validation set, then just fit the model
            self.model = lgb.train(params, lgb_train, num_boost_round=20 )
        else:
            # Define the validation set
            lgb_eval = lgb.Dataset(X_test, label=y_test.ravel(), \
                            categorical_feature=self.categorical_feature, reference=lgb_train )
    
            # Fit the model with the validation set and early stopping
            self.model = lgb.train(params, lgb_train, num_boost_round=20,
                valid_sets=lgb_eval, early_stopping_rounds=5 )
        
    
    def predict(self, X):
        y_hat = self.model.predict( X, num_iteration=gbm.best_iteration)
        return( y_hat )

In [204]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse', 'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

X_train, y_train, X_test, y_test = data_sets[-2]
model = LGBM(params=params, categorical_feature=cat_features )

model.fit(X_train, y_train, X_test, y_test)
y_hat = model.predict(X_test)

print('The rmse of prediction is:', mean_squared_error(y_test, y_hat) ** 0.5)

/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's rmse: 3.55662
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 3.52179
[3]	valid_0's rmse: 3.49066
[4]	valid_0's rmse: 3.46234
[5]	valid_0's rmse: 3.43875
[6]	valid_0's rmse: 3.4106
[7]	valid_0's rmse: 3.38847
[8]	valid_0's rmse: 3.36681
[9]	valid_0's rmse: 3.3488
[10]	valid_0's rmse: 3.33003
[11]	valid_0's rmse: 3.31067
[12]	valid_0's rmse: 3.293
[13]	valid_0's rmse: 3.28196
[14]	valid_0's rmse: 3.27193
[15]	valid_0's rmse: 3.25503
[16]	valid_0's rmse: 3.24482
[17]	valid_0's rmse: 3.23585
[18]	valid_0's rmse: 3.22909
[19]	valid_0's rmse: 3.22107
[20]	valid_0's rmse: 3.22176
Did not meet early stopping. Best iteration is:
[19]	valid_0's rmse: 3.22107
The rmse of prediction is: 3.188448785572161


In [210]:
model_name = 'lightgbm_01'

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse', 'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

base_model = lambda : LGBM(params=params, categorical_feature=cat_features )
fitted_models = fit_model( base_model, model_name, data_sets )

/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Writing results for data set 0. RMSE is 0.8915620991031055


/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Writing results for data set 1. RMSE is 1.0301535214194166


/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Writing results for data set 2. RMSE is 4.212827720482981


/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Writing results for data set 3. RMSE is 3.188448785572161


/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Writing results for data set 4. RMSE is nan


In [ ]:
# Try a forecast with XGBoost
###################################################

from xgboost import XGBRegressor

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-2]

n_estimators = 50
max_depth = 4           # No lower than 3. Increase until performance stops improving
learning_rate = 0.05    # Keep in the range of 0.01 and 0.1
gamma = 5               # Regularization parameter: use value 0, 1, or 5
colsample_bytree = 0.2  # Between 0.3 and 0.8 when dataset has many columns

# Construct the model
model = XGBRegressor( n_estimators=n_estimators, \
                      gamma=gamma, \
                      colsample_bytree=colsample_bytree,\
                      max_depth=max_depth, \
                      learning_rate=learning_rate )

model.fit( X_train, y_train )

# Check the out-of-sample fit for the validation set
yhat = model.predict(X_test)

# Print out the RMSE
rmse = np.sqrt(mean_squared_error( yhat, y_test ))
print( 'RMSE on the validation set: {}'.format(rmse))

In [181]:
# Try a forecast with XGBoost
###################################################

from xgboost import XGBRegressor

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-1]

n_estimators = 50
max_depth = 4           # No lower than 3. Increase until performance stops improving
learning_rate = 0.05    # Keep in the range of 0.01 and 0.1
gamma = 5               # Regularization parameter: use value 0, 1, or 5
colsample_bytree = 0.2  # Between 0.3 and 0.8 when dataset has many columns

# Construct the model
model = XGBRegressor( n_estimators=n_estimators, \
                      gamma=gamma, \
                      colsample_bytree=colsample_bytree,\
                      max_depth=max_depth, \
                      learning_rate=learning_rate )

model.fit( X_train, y_train )

# Check the out-of-sample fit for the validation set
yhat = model.predict(X_test)

output_file = '../forecasts/test/xgboost_01.csv'
df_out = write_forecast_to_file( yhat, X_test, output_file )

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
# Try a forecast with Catboost
###################################################

from catboost import CatBoostRegressor, Pool

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-2]

train_data = Pool( X_train, y_train.ravel(), cat_features=cat_features )
test_data = Pool( X_test, y_test.ravel(), cat_features=cat_features )

# Construct the model
model = CatBoostRegressor(
    random_seed=63,
    iterations=400,
    learning_rate=0.05,
    l2_leaf_reg=100,
    random_strength=1,
    one_hot_max_size=4,
    leaf_estimation_method='Newton',
    eval_metric='RMSE',
    depth=10,
    boosting_type='Plain',
    bootstrap_type='Bernoulli',
    subsample=0.2,
    rsm=0.2,    
    leaf_estimation_iterations=1,
    max_ctr_complexity=4,
    border_count=32    
)

model.fit(
    train_data,
    logging_level='Silent',
    eval_set=test_data,
    plot=True
)

# Check the out-of-sample fit for the validation set
yhat = model.predict(test_data)

# Print out the RMSE
rmse = np.sqrt(mean_squared_error( yhat, y_test ))
print( 'RMSE on the validation set: {}'.format(rmse))

In [ ]:
# Forecast on the test set with Catboost
###################################################

from catboost import CatBoostRegressor, Pool

###################################################
# First use the validation data set
###################################################
X_train, y_train, X_test, y_test = data_sets[-1]

train_data = Pool( X_train, y_train.ravel(), cat_features=cat_features )

# Construct the model
model = CatBoostRegressor(
    random_seed=63,
    iterations=400,
    learning_rate=0.05,
    l2_leaf_reg=100,
    random_strength=1,
    one_hot_max_size=4,
    leaf_estimation_method='Newton',
    eval_metric='RMSE',
    depth=10,
    boosting_type='Plain',
    bootstrap_type='Bernoulli',
    subsample=0.2,
    rsm=0.2,    
    leaf_estimation_iterations=1,
    max_ctr_complexity=4,
    border_count=32    
)

model.fit(
    train_data,
    logging_level='Silent',
    plot=True
)

# Check the out-of-sample fit for the validation set
yhat = model.predict(X_test)

output_file = '../forecasts/test/catboost_05.csv'
df_out = write_forecast_to_file( yhat, X_test, output_file )